# Setup EJECUCIÓN UNICA. 


Import nltk y descargar stopwords de la pestaña de corpus. 

In [1]:

# Descargamos las stopwords desde nltk corpus (Abrira una ventana nueva.)
# Tenemos que ir a la pestaña de CORPUS, buscar la parte de las stopwords y darle descargar.  
if __name__ == '__main__':
    import nltk
    nltk.download()


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Tenemos que importat parser desde tika. Para esto debemos instalar tika usando  
 $pip install tika  
Para la instalación debemos contar con JDK

**NOTA** No es necesario correr la celda de abajo ya que en el proyecto que se envio ya convertimos los pdfs a txt.  
Tampoco es necesario instalar tika a menos que se quiera volver a convertir los pdf en txt. 


In [3]:


if __name__ == '__main__':
    from tika import parser #pip install tika REQUIERE JDK

import warnings
import os
import unidecode #pip install unidecode
import time
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel

Para nuestro proyecto bajamos los PDF de los planes de cada materia de la carrera de computación.  
Esta sección parsea los datos de los pdf a un formato más manejable, archivos de texto (.txt).  
Estos estan en la carpeta "TXT/"

**NOTA**: no es necesario correr la celda de abajo ya que en el proyecto que se envio ya convertimos los pdfs a txt


In [5]:
#no es necesario correr esta sección ya que en el proyecto que se envio ya convertimos los pdfs a txt

directory = "pdf/"

if __name__ == '__main__':
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"): 
            print(filename)
            raw = parser.from_file(os.path.join(directory, filename))
            #print(raw['content'])
            lines = raw['content']

            lines = unidecode.unidecode(lines[77:])
            #print(lines)
            f = open(os.path.join("TXT/", filename[:-3]+ "txt"), 'w')
            f.write(lines)
            f.close()
            time.sleep(.9)
            continue
        else:
            continue


1089.pdf
1323.pdf
1326.pdf
1425.pdf
1427.pdf
1428.pdf
1429.pdf
1506.pdf
1613.pdf
1827.pdf
1828.pdf
1829.pdf
249.pdf
269.pdf
277.pdf
291.pdf
351.pdf
442.pdf
446.pdf
447.pdf
5.pdf
625.pdf
626.pdf
668.pdf
669.pdf
670.pdf
671.pdf
672.pdf
673.pdf
674.pdf
676.pdf
714.pdf
759.pdf
760.pdf
764.pdf
765.pdf
777.pdf
779.pdf
783.pdf
784.pdf
785.pdf
786.pdf
787.pdf
788.pdf
789.pdf
790.pdf
791.pdf
792.pdf
793.pdf
794.pdf
803.pdf
805.pdf
809.pdf
817.pdf
819.pdf
826.pdf
840.pdf


# Aqui empieza el main

El principal problema para la creacion de nuestro recomendador es el Natural Language Processing o procesamiento de lenguaje natural .  
Para extraer caracteristicas y similitudes entre los documentos, vamos a calcular los vectores de palabras de cada uno de estos.  
Los vectores de palabras son representaciones vectoriales de las palabras de un documento.

Para crearlos vamos a utilizar el Term Frequency-Inverse Document Frequency (TF-IDF). Esto nos devolvera una matriz donde cada columna represente una palabra en nuestro vocabulario, y cada columna representara un plan de estudios.


In [28]:
import os
import unidecode #pip install unidecode
import time
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel




directory = "data/planes_materias/TXT/"

materias = {}
#print("hola")
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        #print(filename) 
        f = open(os.path.join(directory, filename), 'r')
        materias[filename[:-4]] = f.read()

#print(materias['1.txt'])


In [6]:
stop_words = stopwords.words('spanish')
my_stop_words = ["CARRERA","MATEMATICO","SERIACION", "INDICATIVA" ,"ANTECEDENTE","MODALIDAD",
                "CURSO","CARACTER", "TEORICAS", "PRACTICAS" ,"CREDITOS", "HORAS", "SEMANA", "SEMESTRE", "CLAVE",
                "FACULTAD", "CIENCIAS", "OBJETIVO","BIBLIOGRAFIA", "BASICA", "COMPLEMENTARIA","SUGERENCIA",           
                "PARA","LA","EVALUACION","DE","LA","ASIGNATURA","Ademas","de","las","calificaciones",
                "en","examenes","y","tareas","se","tomara","en","cuenta","la","participacion","del","alumno",
                "PERFIL","PROFESIOGRAFICO","Matematico","fisico","actuario","o","licenciado","en","ciencias",
                "de","la","computacion","especialista","en","el","area","de","la","asignatura","a","juicio",
                "del","comite","de","asignacion","de", "cursos","UNIDADES", "TEMATICAS","SERIACION" ,
                "INDICATIVA", "ANTECEDENTE", "SUBSECUENTE", "optativo", "Optativas", "VERSIDAD", "Computacion", "el",
                "plan", "y", "programa", "de", "estudios", "de", "la", "licenciatura", "en", "Ciencias", "Computacion",
                 ">Ciencias", 
                ]
stop_words.append(my_stop_words)
TfidfVectorizer(stop_words=stop_words)


TfidfVectorizer(stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...])

In [9]:

def train(intereses):
    #Palabras que no consideraremos.
    stop_words = stopwords.words('spanish')
    my_stop_words = ["CARRERA","MATEMATICO","SERIACION", "INDICATIVA" ,"ANTECEDENTE","MODALIDAD",
    "CURSO","CARACTER", "TEORICAS", "PRACTICAS" ,"CREDITOS", "HORAS", "SEMANA", "SEMESTRE", "CLAVE",
    "FACULTAD", "CIENCIAS", "OBJETIVO","BIBLIOGRAFIA", "BASICA", "COMPLEMENTARIA","SUGERENCIA",           "PARA","LA","EVALUACION","DE","LA","ASIGNATURA","Ademas","de","las","calificaciones",
    "en","examenes","y","tareas","se","tomara","en","cuenta","la","participacion","del","alumno",
    "PERFIL","PROFESIOGRAFICO","Matematico","fisico","actuario","o","licenciado","en","ciencias",
    "de","la","computacion","especialista","en","el","area","de","la","asignatura","a","juicio",
    "del","comite","de","asignacion","de", "cursos","UNIDADES", "TEMATICAS","SERIACION" ,
    "INDICATIVA" ,"ANTECEDENTE", "SUBSECUENTE","optativo","Optativas"]
    stop_words.append(my_stop_words)
    materias[0] = intereses
    #data = pd.DataFrame(materias)
    data = pd.Series(materias).to_frame()
    type(data)
    data = data.rename(columns={0: 'Planes'})
    #data["index"] = pd.to_numeric(data["index"])
    data.index = data.index.map(int)
    data.sort_index(inplace=True)

    #print(data.columns)
    #print(data.index)


    #Define a TF-IDF Vectorizer Object. Remove all spanis stop words such as 'de', 'la', 'que', 'el'
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        tfidf = TfidfVectorizer(stop_words)

    #Construct the required TF-IDF matrix by fitting and transforming the data
    #print(data['Planes'])
    tfidf_matrix = tfidf.fit_transform(data['Planes'])

    #Output the shape of tfidf_matrix
    tfidf_matrix.shape

    #Palabras encontradas en todos los archivos
    tfidf.get_feature_names()[2990:3007]


    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    (cosine_sim.shape)
    #print(cosine_sim[1])
    
    return data, cosine_sim
    

In [10]:
#indices = pd.Series(data.index, index=data.index).drop_duplicates()

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(intereses):
    # Get the index of the movie that matches the title
    data, matriz_similitudes = train(intereses)

    # Get the pairwsie similarity scores of all plans with la lista
    sim_scores = list(enumerate(matriz_similitudes[0]))
    #print(sim_scores)

    # Sort the topics based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #print(sim_scores)
    # Get the scores of the  10most similar topics
    sim_scores = sim_scores[0:11]

    # Get the movie indices
    recomendaciones = [i[0] for i in sim_scores]
    #print(recomendaciones)

    # Return the top 10 most similar topics
    return list(data.iloc[recomendaciones].index[1:])

In [29]:
# Devuelve una lista de 10 recomendaciones de la posicion 5 de los temarios. 

# Poner posicion, no clave.
#1.pdf -> 0
#2.pdf -> 1

"""
recomendacion = get_recommendations("Ecuaciones diferenciales")
recomendacion
"""
"""recomendacion = get_recommendations("topologia ecuaciones diferenciales")
recomendacion
"""
get_recommendations('topologia ecuaciones diferenciales')


[162, 765, 183, 165, 766, 395, 433, 163, 712, 939]

In [94]:
#recomendacion